<a href="https://colab.research.google.com/github/Piyu569/SparkProject02/blob/dev/Partitioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 13.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=179d96f2f00edb9514a44b87dc0ed67d605d720bc64b81aaebdc7a3125e665ae
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
import pyspark

In [7]:
spark = SparkSession.builder.appName("YouSubscription").getOrCreate()

# YouTube DataSet for partitioning

In [69]:
from pyspark.sql.types import LongType, StringType, StructType, StructField ,IntegerType

userDefinedSchema = StructType([
    StructField("Rank", IntegerType(), True),
    StructField("Youtube Channerl", StringType(), True),
    StructField("Subscribers",StringType(),True),
    StructField("Vedio Views", StringType(), True),
    StructField("Vedio Count",StringType(),True),
    StructField("Category" , StringType() , True)
    
])

In [105]:
usersCsvPath = "/content/topSubscribed.csv"

df = (spark
           .read
           .option("header", True)
           .schema(userDefinedSchema)
           .csv(usersCsvPath)
          )

In [106]:
df.show(20,False)
df.printSchema()

+----+--------------------------+-----------+---------------+-----------+----------------------------------------------------------------------------+
|Rank|Youtube Channerl          |Subscribers|Vedio Views    |Vedio Count|Category                                                                    |
+----+--------------------------+-----------+---------------+-----------+----------------------------------------------------------------------------+
|1   |T-Series                  |234,000,000|212,900,271,553|18,515     |Music                                                                       |
|2   |YouTube Movies            |161,000,000|0              |0          |Film & Animation                                                            |
|3   |Cocomelon - Nursery Rhymes|152,000,000|149,084,178,448|846        |Education                                                                   |
|4   |SET India                 |150,000,000|137,828,094,104|103,200    |Shows                

In [107]:
df.count()

1000

In [108]:
df.rdd.getNumPartitions()

1

In [109]:
print(spark.sparkContext.defaultMinPartitions)

2


In [110]:
print(spark.sparkContext.defaultParallelism)

2


In [111]:
rp = df.repartition(8)

In [112]:
rp.rdd.getNumPartitions()

8

In [113]:
co = df.coalesce(8)

In [114]:
co.rdd.getNumPartitions()

1

# which channel category having most subscribers and sort subscribers in decreasing order

In [130]:
from pyspark.sql.functions import avg,sum,col,sort_array,regexp_replace
strtoint = (rp.withColumn("Subscribers",regexp_replace(col("Subscribers"),",","").cast("integer"))
              .groupBy("Category")
              .agg(sum("Subscribers").alias("total_subs"))
              .orderBy(col("total_subs").desc())
).show(20,False)

+----------------------------------------------------------------------------+----------+
|Category                                                                    |total_subs|
+----------------------------------------------------------------------------+----------+
|Music                                                                       |5224800000|
|Entertainment                                                               |4768200000|
|People & Blogs                                                              |2558500000|
|Gaming                                                                      |1873900000|
|Film & Animation                                                            |1275200000|
|Comedy                                                                      |1256400000|
|Education                                                                   |1133600000|
|https://us.youtubers.me/global/all/top-1000-most_subscribed-youtube-channels|793400000 |
|Howto & S

# Which music channel having the most subs and sort them in decending order

In [142]:
from pyspark.sql.functions import when
music = (co.withColumn("Subscribers",regexp_replace(col("Subscribers"),",","").cast("integer"))
           .filter(df["Category"] == "Music")
           .orderBy(col("Subscribers").desc())       
).show()

+----+--------------------+-----------+---------------+-----------+--------+
|Rank|    Youtube Channerl|Subscribers|    Vedio Views|Vedio Count|Category|
+----+--------------------+-----------+---------------+-----------+--------+
|   1|            T-Series|  234000000|212,900,271,553|     18,515|   Music|
|  13|   Zee Music Company|   91600000| 53,352,597,784|      7,530|   Music|
|  19|           BANGTANTV|   73100000| 19,240,057,894|      2,090|   Music|
|  20|       Justin Bieber|   70800000| 29,598,652,357|        249|   Music|
|  21|         HYBE LABELS|   69600000| 25,947,702,825|      1,070|   Music|
|  22|     Canal KondZilla|   66300000| 37,179,407,050|      2,383|   Music|
|  25|Shemaroo Filmi Gaane|   63500000| 29,992,048,346|      8,464|   Music|
|  30|T-Series Bhakti S...|   57300000| 27,004,807,684|     24,900|   Music|
|  31|       Tips Official|   56300000| 30,741,513,673|      4,102|   Music|
|  33|          Wave Music|   55700000| 38,819,471,815|     18,970|   Music|

# Channel with max vedios 

In [163]:
from pyspark.sql.functions import max , desc
mx_vedios = (rp.withColumn("Vedio Count",regexp_replace(col("Vedio Count"),",","").cast("integer"))
               .sort(desc("Vedio Count"))        
  ).show(1)


+----+----------------+-----------+--------------+-----------+---------------+
|Rank|Youtube Channerl|Subscribers|   Vedio Views|Vedio Count|       Category|
+----+----------------+-----------+--------------+-----------+---------------+
| 821|        GMA News| 12,800,000|11,220,255,933|     342802|News & Politics|
+----+----------------+-----------+--------------+-----------+---------------+
only showing top 1 row



# In Gaming Catgory Channel with max vedios 

In [166]:
maxVedios = (rp.withColumn("Vedio Count",regexp_replace(col("Vedio Count"),",","").cast("integer"))
               .filter(rp["Category"] == "Gaming")
               .orderBy(col("Vedio Count").desc())
).show(1)


+----+----------------+-----------+--------------+-----------+--------+
|Rank|Youtube Channerl|Subscribers|   Vedio Views|Vedio Count|Category|
+----+----------------+-----------+--------------+-----------+--------+
| 464|             IGN| 17,200,000|15,298,075,378|     152548|  Gaming|
+----+----------------+-----------+--------------+-----------+--------+
only showing top 1 row



# Top 5 youtube channel:

In [167]:
topfive = (co.withColumn("Subscribers",regexp_replace(col("Subscribers"),",","").cast("integer"))
             .sort(desc("Subscribers"))                 
     ).show(5)

+----+--------------------+-----------+---------------+-----------+----------------+
|Rank|    Youtube Channerl|Subscribers|    Vedio Views|Vedio Count|        Category|
+----+--------------------+-----------+---------------+-----------+----------------+
|   1|            T-Series|  234000000|212,900,271,553|     18,515|           Music|
|   2|      YouTube Movies|  161000000|              0|          0|Film & Animation|
|   3|Cocomelon - Nurse...|  152000000|149,084,178,448|        846|       Education|
|   4|           SET India|  150000000|137,828,094,104|    103,200|           Shows|
|   5|             MrBeast|  128000000| 21,549,128,785|        733|   Entertainment|
+----+--------------------+-----------+---------------+-----------+----------------+
only showing top 5 rows



# Bottom 5 youTube Channel

In [175]:
from pyspark.sql.functions import asc
bottomfive = (co.withColumn("Subscribers",regexp_replace(col("Subscribers"),",","").cast("integer"))
             .sort(asc("Subscribers"))             
     ).show(5)

+----+----------------+-----------+-------------+-----------+--------------+
|Rank|Youtube Channerl|Subscribers|  Vedio Views|Vedio Count|      Category|
+----+----------------+-----------+-------------+-----------+--------------+
| 999|       Painzeiro|   11400000|2,143,444,281|        946|People & Blogs|
| 998|         YtCrash|   11400000|    3,825,084|          1|     Education|
|1000|       Oum Walid|   11400000|2,480,064,722|      1,598|People & Blogs|
| 981|         Airrack|   11500000|1,542,774,923|        159| Entertainment|
| 986|       Brancoala|   11500000|8,465,443,935|      1,768| Entertainment|
+----+----------------+-----------+-------------+-----------+--------------+
only showing top 5 rows

